# BioMapAI Tutorial

This tutorial will guide you through:
1. **Environment Setup**  
2. **Data Preparation**  
3. **Model Initialization and Training**  
4. **Prediction and Evaluation**  
5. **Score Conversion and Weight Adjustment**  

By the end, you should be able to train your own model on custom data.


## 1. Environment Setup

1. **Download or Clone** the repository/project that contains:
   - **DNN.py** (the script with model classes and methods)
   - **example_data/** (containing your `train_data.csv` and `test_data.csv`)
   - A script or notebook for your tutorial (this file)

2. **Install Required Libraries** (if you haven’t already):
```bash
pip install numpy pandas tensorflow
```

3. **Import modules** in Python:
```python
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import importlib.util
import os

# Clear any previous TensorFlow session
tf.keras.backend.clear_session()
```

4. **Function to import `DNN.py`**:
```python
def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module
```


## 2. Data Preparation

Let’s assume you have:
- A **training dataset** named `train_data.csv`
- A **test dataset** named `test_data.csv`

Each row might look like:

| y0 (label) | Y1 | Y2 | ... | Y12 | X1 | X2 | ... | Xn |
|------------|----|----|-----|-----|----|----|-----|----|
| 0 or 1     | …  | …  | …   | …   | …  | …  | …   | …  |

Where:
- **y0** is your single-class label (e.g., 0 or 1)
- **Y** columns (Y1–Y12) are intermediate targets
- **X** columns (X1–Xn) are input features

### Load Your Data
```python
# 1) Import the DNN module
DNN = import_module_with_full_path("DNN.py")

# 2) Load training data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]       # Single-class label
Y = data.iloc[:, 1:13]     # Intermediate target columns
X = data.iloc[:, 13:]      # Input features

# 3) Load test data
test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]
```


## 3. Model Initialization and Training

### 3.1 **OmicScoreModel**
The **OmicScoreModel** is trained to predict the intermediate targets `Y`.

```python
model, model_history = DNN.OmicScoreModel(
    epochs=500, 
    optimizer=tf.keras.optimizers.Adam(0.0005), 
    batch_size=64, 
    kernel_regularizer=tf.keras.regularizers.L2(0.008), 
    model_name="example"
).train(X, Y)
```
- **epochs**: Number of training iterations.
- **optimizer**: Choice of optimizer and learning rate.
- **batch_size**: Batch size for training.
- **kernel_regularizer**: Regularization to avoid overfitting.
- **model_name**: Useful if you want to save or track multiple models.

### 3.2 Monitoring the Training History
```python
print(model_history.history.keys())
# You can plot loss curves, accuracy, etc.
```


## 4. Prediction and Evaluation

### 4.1 Predict Intermediate Targets `Y`
```python
Y_pred = DNN.OmicScoreModel().predict(model, X_test)
```
This outputs the intermediate predictions for each of the 12 `Y` columns.

### 4.2 Evaluate Intermediate Target Predictions
Compute the Mean Squared Error (MSE) on `Y_test`:
```python
mse = DNN.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)
```


## 5. Score Conversion and Weight Adjustment

### 5.1 Build a Layer for Converting `Y` to `y0`
```python
Y_to_y_model = DNN.ScoreLayer().build_model(Y, y0)
```
This creates a secondary model that maps the 12-dimensional `Y` to a single label or score.

### 5.2 Predict Final Labels `y0`
```python
y_pred = DNN.ScoreYModel(model, Y_to_y_model).predict(X_test)
```
### 5.3 Evaluate Final Model
```python
loss, accuracy = DNN.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Final Model Loss:", loss)
print("Final Model Accuracy:", accuracy)
```

### 5.4 Adjusting Weights (Optional)
If you want to fine-tune how the intermediate predictions map to the final label:
```python
Y_to_y_model_adjust = DNN.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()
```
Then re-predict and re-evaluate:
```python
y_pred = DNN.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = DNN.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)
```


## Putting It All Together

Below is a compact version of the entire flow. You can copy this code into its own file (e.g., `train_and_eval.py`) or run it as a Jupyter cell.


In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import random
import tensorflow as tf
tf.keras.backend.clear_session()

import importlib.util
import os

def import_module_with_full_path(file_path):
    base_filename = os.path.basename(file_path)
    module_name = os.path.splitext(base_filename)[0]
    module_spec = importlib.util.spec_from_file_location(module_name, file_path)
    imported_module = importlib.util.module_from_spec(module_spec)
    module_spec.loader.exec_module(imported_module)
    return imported_module

# 1) Import DNN module
DNN = import_module_with_full_path("DNN.py")

# 2) Load data
data = pd.read_csv("example_data/train_data.csv", index_col=0)
y0 = data.iloc[:, 0]
Y = data.iloc[:, 1:13]
X = data.iloc[:, 13:]

test_data = pd.read_csv("example_data/test_data.csv", index_col=0)
y0_test = test_data.iloc[:, 0]
Y_test = test_data.iloc[:, 1:13]
X_test = test_data.iloc[:, 13:]

# 3) Train OmicScoreModel
model, model_history = DNN.OmicScoreModel(
    epochs=500,
    optimizer=tf.keras.optimizers.Adam(0.0005),
    batch_size=64,
    kernel_regularizer=tf.keras.regularizers.L2(0.008),
    model_name="example"
).train(X, Y)

# 4) Predict and evaluate intermediate targets
Y_pred = DNN.OmicScoreModel().predict(model, X_test)
mse = DNN.OmicScoreModel().score(model, X_test, Y_test)
print("Intermediate Target MSE:", mse)

# 5) Build layer to convert Y --> y0
Y_to_y_model = DNN.ScoreLayer().build_model(Y, y0)

# 6) Predict final output
y_pred = DNN.ScoreYModel(model, Y_to_y_model).predict(X_test)

# 7) Evaluate final output
loss, accuracy = DNN.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Initial Final Model Loss:", loss)
print("Initial Final Model Accuracy:", accuracy)

# 8) (Optional) Adjust weights
Y_to_y_model_adjust = DNN.WeightsAdjust(model, X, Y, Y_to_y_model).adjust_score_weight()

# 9) Predict & Evaluate again
y_pred = DNN.ScoreYModel(model, Y_to_y_model).predict(X_test)
loss, accuracy = DNN.ScoreYModel(model, Y_to_y_model).evaluate(X_test, y0_test)
print("Adjusted Final Model Loss:", loss)
print("Adjusted Final Model Accuracy:", accuracy)


2025-01-14 16:02:28.456013: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 16:02:29.474230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/500
3/3 [==============================] - 5s 12ms/step - loss: 8.1695 - out0_loss: 0.2345 - out1_loss: 2.1806 - out2_loss: 1.2860 - out3_loss: 0.2750 - out4_loss: 0.2578 - out5_loss: 0.3165 - out6_loss: 0.4153 - out7_loss: 0.1414 - out8_loss: 0.2280 - out9_loss: 0.1927 - out10_loss: 0.1522 - out11_loss: 0.2653 - out0_mse: 0.5564 - out1_accuracy: 0.0743 - out2_accuracy: 0.3943 - out3_mse: 0.5499 - out4_mse: 0.8117 - out5_mse: 0.8376 - out6_mse: 1.1896 - out7_mse: 0.2831 - out8_mse: 0.5984 - out9_mse: 0.3912 - out10_mse: 0.3207 - out11_mse: 0.6385
Epoch 2/500
3/3 [==============================] - 0s 8ms/step - loss: 8.0339 - out0_loss: 0.1694 - out1_loss: 2.1534 - out2_loss: 1.3137 - out3_loss: 0.2730 - out4_loss: 0.2683 - out5_loss: 0.3056 - out6_loss: 0.4121 - out7_loss: 0.1366 - out8_loss: 0.2273 - out9_loss: 0.2097 - out10_loss: 0.1266 - out11_loss: 0.2220 - out0_mse: 0.3700 - out1_accuracy: 0.0800 - out2_accuracy: 0.3943 - out3_mse: 0.5460 - out4_mse: 0.7951 - out5_mse: 0.

3/3 [==============================] - 0s 7ms/step - loss: 7.0311 - out0_loss: 0.1416 - out1_loss: 1.8820 - out2_loss: 1.2597 - out3_loss: 0.2603 - out4_loss: 0.1479 - out5_loss: 0.2281 - out6_loss: 0.2985 - out7_loss: 0.1302 - out8_loss: 0.1394 - out9_loss: 0.1681 - out10_loss: 0.1083 - out11_loss: 0.1447 - out0_mse: 0.3043 - out1_accuracy: 0.1600 - out2_accuracy: 0.4286 - out3_mse: 0.5207 - out4_mse: 0.3912 - out5_mse: 0.5004 - out6_mse: 0.7303 - out7_mse: 0.2605 - out8_mse: 0.2849 - out9_mse: 0.3401 - out10_mse: 0.2166 - out11_mse: 0.3396
Epoch 16/500
3/3 [==============================] - 0s 7ms/step - loss: 7.0639 - out0_loss: 0.1713 - out1_loss: 1.8450 - out2_loss: 1.2586 - out3_loss: 0.2573 - out4_loss: 0.1352 - out5_loss: 0.2369 - out6_loss: 0.2626 - out7_loss: 0.1334 - out8_loss: 0.1708 - out9_loss: 0.1710 - out10_loss: 0.1330 - out11_loss: 0.1735 - out0_mse: 0.3752 - out1_accuracy: 0.1543 - out2_accuracy: 0.4743 - out3_mse: 0.5147 - out4_mse: 0.3458 - out5_mse: 0.5272 - out6_

3/3 [==============================] - 0s 8ms/step - loss: 6.4159 - out0_loss: 0.1441 - out1_loss: 1.7183 - out2_loss: 1.2021 - out3_loss: 0.2464 - out4_loss: 0.1046 - out5_loss: 0.1593 - out6_loss: 0.2258 - out7_loss: 0.1178 - out8_loss: 0.1230 - out9_loss: 0.1526 - out10_loss: 0.1122 - out11_loss: 0.0790 - out0_mse: 0.3074 - out1_accuracy: 0.2686 - out2_accuracy: 0.5143 - out3_mse: 0.4928 - out4_mse: 0.2224 - out5_mse: 0.3491 - out6_mse: 0.5600 - out7_mse: 0.2358 - out8_mse: 0.2625 - out9_mse: 0.3070 - out10_mse: 0.2263 - out11_mse: 0.1609
Epoch 30/500
3/3 [==============================] - 0s 7ms/step - loss: 6.2798 - out0_loss: 0.1467 - out1_loss: 1.6523 - out2_loss: 1.1301 - out3_loss: 0.2468 - out4_loss: 0.1149 - out5_loss: 0.1657 - out6_loss: 0.2020 - out7_loss: 0.1221 - out8_loss: 0.1183 - out9_loss: 0.1563 - out10_loss: 0.1109 - out11_loss: 0.0894 - out0_mse: 0.3246 - out1_accuracy: 0.2571 - out2_accuracy: 0.5429 - out3_mse: 0.4935 - out4_mse: 0.2453 - out5_mse: 0.3571 - out6_

3/3 [==============================] - 0s 7ms/step - loss: 5.7717 - out0_loss: 0.1291 - out1_loss: 1.5069 - out2_loss: 1.1000 - out3_loss: 0.2194 - out4_loss: 0.0730 - out5_loss: 0.1420 - out6_loss: 0.1474 - out7_loss: 0.0944 - out8_loss: 0.1021 - out9_loss: 0.1414 - out10_loss: 0.1102 - out11_loss: 0.0608 - out0_mse: 0.2747 - out1_accuracy: 0.3143 - out2_accuracy: 0.5943 - out3_mse: 0.4388 - out4_mse: 0.1474 - out5_mse: 0.2944 - out6_mse: 0.3127 - out7_mse: 0.1888 - out8_mse: 0.2058 - out9_mse: 0.2868 - out10_mse: 0.2216 - out11_mse: 0.1233
Epoch 44/500
3/3 [==============================] - 0s 7ms/step - loss: 5.7809 - out0_loss: 0.1177 - out1_loss: 1.5373 - out2_loss: 1.0288 - out3_loss: 0.2152 - out4_loss: 0.0663 - out5_loss: 0.1457 - out6_loss: 0.1860 - out7_loss: 0.0990 - out8_loss: 0.1044 - out9_loss: 0.1389 - out10_loss: 0.1152 - out11_loss: 0.0873 - out0_mse: 0.2439 - out1_accuracy: 0.3714 - out2_accuracy: 0.5829 - out3_mse: 0.4304 - out4_mse: 0.1335 - out5_mse: 0.2960 - out6_

...

3/3 [==============================] - 0s 7ms/step - loss: 1.5643 - out0_loss: 0.0203 - out1_loss: 0.3307 - out2_loss: 0.5073 - out3_loss: 0.0229 - out4_loss: 0.0180 - out5_loss: 0.0204 - out6_loss: 0.0219 - out7_loss: 0.0118 - out8_loss: 0.0147 - out9_loss: 0.0094 - out10_loss: 0.0164 - out11_loss: 0.0207 - out0_mse: 0.0407 - out1_accuracy: 0.9143 - out2_accuracy: 0.7943 - out3_mse: 0.0457 - out4_mse: 0.0360 - out5_mse: 0.0409 - out6_mse: 0.0437 - out7_mse: 0.0237 - out8_mse: 0.0294 - out9_mse: 0.0187 - out10_mse: 0.0328 - out11_mse: 0.0414
Epoch 478/500
3/3 [==============================] - 0s 7ms/step - loss: 1.5540 - out0_loss: 0.0218 - out1_loss: 0.3117 - out2_loss: 0.5155 - out3_loss: 0.0270 - out4_loss: 0.0178 - out5_loss: 0.0211 - out6_loss: 0.0207 - out7_loss: 0.0133 - out8_loss: 0.0127 - out9_loss: 0.0095 - out10_loss: 0.0158 - out11_loss: 0.0186 - out0_mse: 0.0437 - out1_accuracy: 0.8971 - out2_accuracy: 0.8114 - out3_mse: 0.0540 - out4_mse: 0.0356 - out5_mse: 0.0423 - out6

3/3 [==============================] - 0s 7ms/step - loss: 1.5147 - out0_loss: 0.0207 - out1_loss: 0.2964 - out2_loss: 0.5144 - out3_loss: 0.0254 - out4_loss: 0.0165 - out5_loss: 0.0213 - out6_loss: 0.0196 - out7_loss: 0.0128 - out8_loss: 0.0121 - out9_loss: 0.0097 - out10_loss: 0.0139 - out11_loss: 0.0212 - out0_mse: 0.0414 - out1_accuracy: 0.8857 - out2_accuracy: 0.8171 - out3_mse: 0.0508 - out4_mse: 0.0329 - out5_mse: 0.0427 - out6_mse: 0.0392 - out7_mse: 0.0255 - out8_mse: 0.0242 - out9_mse: 0.0194 - out10_mse: 0.0277 - out11_mse: 0.0423
Epoch 492/500
3/3 [==============================] - 0s 7ms/step - loss: 1.5733 - out0_loss: 0.0190 - out1_loss: 0.3241 - out2_loss: 0.5469 - out3_loss: 0.0219 - out4_loss: 0.0200 - out5_loss: 0.0206 - out6_loss: 0.0194 - out7_loss: 0.0126 - out8_loss: 0.0139 - out9_loss: 0.0097 - out10_loss: 0.0161 - out11_loss: 0.0198 - out0_mse: 0.0381 - out1_accuracy: 0.9029 - out2_accuracy: 0.8000 - out3_mse: 0.0437 - out4_mse: 0.0400 - out5_mse: 0.0412 - out6

6/6 [==============================] - 0s 1ms/step - loss: 0.1976 - accuracy: 0.9714
Epoch 24/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1914 - accuracy: 0.9714
Epoch 25/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1855 - accuracy: 0.9771
Epoch 26/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1802 - accuracy: 0.9771
Epoch 27/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1749 - accuracy: 0.9771
Epoch 28/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1710 - accuracy: 0.9771
Epoch 29/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1658 - accuracy: 0.9771
Epoch 30/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1619 - accuracy: 0.9771
Epoch 31/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1582 - accuracy: 0.9771
Epoch 32/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1546 - accuracy: 0.9771
Epoch 33/200
6/6 [===============

6/6 [==============================] - 0s 1ms/step - loss: 0.0855 - accuracy: 0.9829
Epoch 107/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0855 - accuracy: 0.9829
Epoch 108/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0853 - accuracy: 0.9829
Epoch 109/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0851 - accuracy: 0.9829
Epoch 110/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0848 - accuracy: 0.9829
Epoch 111/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0847 - accuracy: 0.9829
Epoch 112/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0847 - accuracy: 0.9829
Epoch 113/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0843 - accuracy: 0.9829
Epoch 114/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0840 - accuracy: 0.9829
Epoch 115/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0840 - accuracy: 0.9829
Epoch 116/200
6/6 [=====

6/6 [==============================] - 0s 1ms/step - loss: 0.0765 - accuracy: 0.9829
Epoch 189/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0763 - accuracy: 0.9829
Epoch 190/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0763 - accuracy: 0.9829
Epoch 191/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0762 - accuracy: 0.9829
Epoch 192/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0762 - accuracy: 0.9829
Epoch 193/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0762 - accuracy: 0.9829
Epoch 194/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0760 - accuracy: 0.9829
Epoch 195/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0759 - accuracy: 0.9829
Epoch 196/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0759 - accuracy: 0.9829
Epoch 197/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0759 - accuracy: 0.9829
Epoch 198/200
6/6 [=====

## Conclusion
You have now:
1. **Loaded and prepared** your data
2. **Trained** the OmicScoreModel on intermediate targets
3. **Transformed** those intermediate predictions to final labels
4. **Evaluated** and **(optionally) adjusted** model weights

Feel free to experiment with different architectures, hyperparameters (learning rate, batch size, etc.), or data splits. Modify **DNN.py** to suit your specific needs.  

**Happy modeling!**